<a href="https://colab.research.google.com/github/hammerheaddown/ESP-32S3-Apex-AI/blob/main/Fast_Whisper_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install faster-whisper
!pip install fastapi nest-asyncio pyngrok uvicorn

## Server

In [ ]:
from google.colab import userdata
NGROK_TOKEN = userdata.get('NGROK_TOKEN')
!ngrok config add-authtoken {NGROK_TOKEN}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import StreamingResponse
from pyngrok import ngrok
import nest_asyncio
import uvicorn
from faster_whisper import WhisperModel
from io import BytesIO
import time

app = FastAPI()

# Load the Whisper model
model_name = "Systran/faster-distil-whisper-medium.en"
model_name = "distil-large-v3"
model = WhisperModel(model_name, device="cuda", compute_type="float16")

@app.post("/transcribe_stream")
async def transcribe_stream(audio: UploadFile = File(...), initial_prompt: str = Form("")):
    # Save the uploaded audio file to a BytesIO object
    audio_bytes = BytesIO(await audio.read())

    # Transcribe the audio file
    segments, info = model.transcribe(audio_bytes, beam_size=5, language="en", condition_on_previous_text=False, word_timestamps=True, initial_prompt=initial_prompt)

    # Stream the transcription segments
    async def stream_segments():
        for segment in segments:
            yield f"{segment.text}\n"

    return StreamingResponse(stream_segments(), media_type="text/plain")

@app.post("/transcribe")
async def transcribe(audio: UploadFile = File(...), initial_prompt: str = Form("")):
    # Save the uploaded audio file to a BytesIO object
    audio_bytes = BytesIO(await audio.read())

    # Transcribe the audio file
    start = time.time()
    segments, info = model.transcribe(audio_bytes, beam_size=5, language="en", condition_on_previous_text=False, word_timestamps=True, initial_prompt=initial_prompt)

    # Convert segments to a list
    segments = list(segments)
    end = time.time()
    print("Transcription time: %.2fs" % (end - start))

    json_segments = []

    for s in segments:
        sg = {}
        for field in ['id', 'seek', 'start', 'end', 'text', 'words']:
            sg[field] = getattr(s, field)
            sg['words'] = []
            for word in s.words:
                sg['words'].append({field: getattr(word, field) for field in ['start', 'end', 'word']})

        json_segments.append(sg)

    return {"segments": json_segments}

reserved_domain = "colab.ngrok.pro"
ngrok_tunnel = ngrok.connect(8000, hostname=reserved_domain)
print(f"Public URL: https://{reserved_domain}")

# ngrok_tunnel = ngrok.connect(8000)
# print('Public URL:', ngrok_tunnel.public_url)

nest_asyncio.apply()

uvicorn.run(app, host='0.0.0.0', port=8000)

model.bin:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

vocabulary.json:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

INFO:     Started server process [196]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://colab.ngrok.pro
INFO:     2001:f40:910:1983:e90c:d5b9:3247:5d6a:0 - "POST /transcribe_stream HTTP/1.1" 200 OK


INFO:     2001:f40:910:1983:e90c:d5b9:3247:5d6a:0 - "POST /transcribe_stream HTTP/1.1" 200 OK
INFO:     2001:f40:910:1983:e90c:d5b9:3247:5d6a:0 - "POST /transcribe_stream HTTP/1.1" 200 OK
INFO:     2001:f40:910:1983:e90c:d5b9:3247:5d6a:0 - "POST /transcribe_stream HTTP/1.1" 200 OK
INFO:     2001:f40:910:1983:e90c:d5b9:3247:5d6a:0 - "POST /transcribe_stream HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 399, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 164, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dis

INFO:     2001:f40:910:1983:e90c:d5b9:3247:5d6a:0 - "POST /transcribe_stream HTTP/1.1" 200 OK
INFO:     2001:f40:910:1983:e90c:d5b9:3247:5d6a:0 - "POST /transcribe_stream HTTP/1.1" 200 OK
INFO:     2001:f40:910:1983:e90c:d5b9:3247:5d6a:0 - "POST /transcribe_stream HTTP/1.1" 200 OK
INFO:     2001:f40:910:1983:e90c:d5b9:3247:5d6a:0 - "POST /transcribe_stream HTTP/1.1" 200 OK
INFO:     2001:f40:910:1983:e90c:d5b9:3247:5d6a:0 - "POST /transcribe_stream HTTP/1.1" 200 OK
INFO:     2001:f40:910:1983:e90c:d5b9:3247:5d6a:0 - "POST /transcribe_stream HTTP/1.1" 200 OK
INFO:     2001:f40:910:1983:e90c:d5b9:3247:5d6a:0 - "POST /transcribe_stream HTTP/1.1" 200 OK
